## Assignment Cassandra

## Submitted by Aarsha Joji

### Import packages

In [60]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [61]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [62]:
session.execute("DROP KEYSPACE IF EXISTS bd23")

In [63]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


### Create a keyspace - bd23

In [64]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS bd23 WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

In [65]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


### Create a inventory table for testing

In [66]:
session.execute("""
CREATE TABLE IF NOT EXISTS bd23.inventory ( 
    sku TEXT, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT, 
    PRIMARY KEY(sku)
);
""")

### Load data

In [67]:
df = pd.read_csv('inventory.csv')
df

,sku,prod_name,description,warehouse_num
0,'SKU1','Product1','DESC1',123
1,'SKU2','Product2','DESC2',125
2,'SKU3','Product3','DESC3',123
3,'SKU4','Product4','DESC4',126
4,'SKU5','Product5','DESC5',123
5,'SKU6','Product6','DESC6',127


In [68]:
for index, row in df.iterrows():
    print(f"sku = {row.sku}, name ={row.prod_name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

sku = 'SKU1', name ='Product1', description = 'DESC1', warehouse_num = 123
sku = 'SKU2', name ='Product2', description = 'DESC2', warehouse_num = 125
sku = 'SKU3', name ='Product3', description = 'DESC3', warehouse_num = 123
sku = 'SKU4', name ='Product4', description = 'DESC4', warehouse_num = 126
sku = 'SKU5', name ='Product5', description = 'DESC5', warehouse_num = 123
sku = 'SKU6', name ='Product6', description = 'DESC6', warehouse_num = 127


In [69]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO bd23.inventory (sku, name, description, warehouse_num)     
        VALUES ({row.sku}, {row.prod_name}, {row.description}, {row.warehouse_num});
        """
    )
    session.execute(f"""
        INSERT INTO bd23.inventory (sku, name, description, warehouse_num)     
        VALUES ({row.sku}, {row.prod_name}, {row.description}, {row.warehouse_num});
        """
    )


        INSERT INTO bd23.inventory (sku, name, description, warehouse_num)     
        VALUES ('SKU1', 'Product1', 'DESC1', 123);
        

        INSERT INTO bd23.inventory (sku, name, description, warehouse_num)     
        VALUES ('SKU2', 'Product2', 'DESC2', 125);
        

        INSERT INTO bd23.inventory (sku, name, description, warehouse_num)     
        VALUES ('SKU3', 'Product3', 'DESC3', 123);
        

        INSERT INTO bd23.inventory (sku, name, description, warehouse_num)     
        VALUES ('SKU4', 'Product4', 'DESC4', 126);
        

        INSERT INTO bd23.inventory (sku, name, description, warehouse_num)     
        VALUES ('SKU5', 'Product5', 'DESC5', 123);
        

        INSERT INTO bd23.inventory (sku, name, description, warehouse_num)     
        VALUES ('SKU6', 'Product6', 'DESC6', 127);
        


In [70]:
rows = session.execute("select (sku, name, description, warehouse_num) from bd23.inventory")
for row in rows:
    print(f"sku={row[0][0]}, prod_name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")

sku=SKU1, prod_name=Product1, description=DESC1, warehouse_num=123
sku=SKU2, prod_name=Product2, description=DESC2, warehouse_num=125
sku=SKU5, prod_name=Product5, description=DESC5, warehouse_num=123
sku=SKU4, prod_name=Product4, description=DESC4, warehouse_num=126
sku=SKU6, prod_name=Product6, description=DESC6, warehouse_num=127
sku=SKU3, prod_name=Product3, description=DESC3, warehouse_num=123


In [71]:
rows = session.execute("select (sku, name, description, warehouse_num) from bd23.inventory where warehouse_num=123 ALLOW FILTERING")
for row in rows:
    print(f"sku={row[0][0]}, prod_name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")

sku=SKU1, prod_name=Product1, description=DESC1, warehouse_num=123
sku=SKU5, prod_name=Product5, description=DESC5, warehouse_num=123
sku=SKU3, prod_name=Product3, description=DESC3, warehouse_num=123
